## A/B Testing simulation with Hypothesis Testing

- Introduction on tests: test on the mean, two sample t test
- A/B Testing simulation

### Test on the mean

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [4]:
np.random.seed(6)
pop_ages_1 = stats.poisson.rvs(loc = 18, mu = 35, size =150000)
pop_ages_2 = stats.poisson.rvs(loc = 18, mu = 10, size =100000)
pop_ages = np.concatenate((pop_ages_1, pop_ages_2))

sample_ages_1 = stats.poisson.rvs(loc = 18, mu = 30, size =30)
sample_ages_2 = stats.poisson.rvs(loc = 18, mu = 10, size =20)
sample_ages = np.concatenate((sample_ages_1, sample_ages_2))

In [7]:
print(pop_ages.mean(), sample_ages.mean()) #different, but does that mean that we can say the same for the original population?

43.000112 39.26


In [8]:
stats.ttest_1samp(a = sample_ages, popmean = pop_ages.mean()) #pvalue < 0.05 -> we reject Ho and hence the two are different!

Ttest_1sampResult(statistic=-2.5742714883655027, pvalue=0.013118685425061678)

In [11]:
#we can also check the value of t-statistic by calculating it manually and comparing it to the alpha = 0.05:
t_stat = (sample_ages.mean() - pop_ages.mean()) / (sample_ages.std()/np.sqrt(len(sample_ages)))
print(t_stat)

-2.6004068943406193


In [15]:
t_alpha = stats.t.ppf(q = 0.025, df = 50 - 1) #alpha / 2 because two tails test
print(t_alpha)

-2.0095752344892093


In [19]:
#finally the confidence interval with 95% confidence level
stats.t.interval(0.95, df = 49, loc = sample_ages.mean(), scale = (sample_ages.std()/np.sqrt(len(sample_ages))))
#you see that true pop mean is outside this interval

(36.369669080722176, 42.15033091927782)

### Two sample t test

In [29]:
np.random.seed(6)
sample_ages_3 = stats.poisson.rvs(loc = 18, mu = 33, size =30)
sample_ages_4 = stats.poisson.rvs(loc = 18, mu = 13, size =20)
sample_ages_b = np.concatenate((sample_ages_3, sample_ages_4))

In [30]:
print(sample_ages.mean(),sample_ages_b.mean())

39.26 43.34


In [32]:
"""
we run indepented t test because from two different samples. you would run paired t test if from same sample just different
point in time (ttest_rel)
"""
stats.ttest_ind(a = sample_ages, b = sample_ages_b, equal_var = False) #pvalue > 0.05 not enough to reject Ho. so they are same

Ttest_indResult(statistic=-1.8900367465301249, pvalue=0.061734324011242535)

### A/B Testing Simulation

In [33]:
import scipy.stats as scs
import pandas as pd
# import numpy as np


def generate_data(N_A, N_B, p_A, p_B, days=None, control_label='A',
                  test_label='B'):
    """Returns a pandas dataframe with fake CTR data
    Example:
    Parameters:
        N_A (int): sample size for control group
        N_B (int): sample size for test group
            Note: final sample size may not match N_A provided because the
            group at each row is chosen at random (50/50).
        p_A (float): conversion rate; conversion rate of control group
        p_B (float): conversion rate; conversion rate of test group
        days (int): optional; if provided, a column for 'ts' will be included
            to divide the data in chunks of time
            Note: overflow data will be included in an extra day
        control_label (str)
        test_label (str)
    Returns:
        df (df)
    """

    # initiate empty container
    data = []

    # total amount of rows in the data
    N = N_A + N_B

    # distribute events based on proportion of group size
    group_bern = scs.bernoulli(N_A / (N_A + N_B))

    # initiate bernoulli distributions from which to randomly sample
    A_bern = scs.bernoulli(p_A)
    B_bern = scs.bernoulli(p_B)

    for idx in range(N):
        # initite empty row
        row = {}
        # for 'ts' column
        if days is not None:
            if type(days) == int:
                row['ts'] = idx // (N // days)
            else:
                raise ValueError("Provide an integer for the days parameter.")
        # assign group based on 50/50 probability
        row['group'] = group_bern.rvs()

        if row['group'] == 0:
            # assign conversion based on provided parameters
            row['converted'] = A_bern.rvs()
        else:
            row['converted'] = B_bern.rvs()
        # collect row into data container
        data.append(row)

    # convert data into pandas dataframe
    df = pd.DataFrame(data)

    # transform group labels of 0s and 1s to user-defined group labels
    df['group'] = df['group'].apply(
        lambda x: control_label if x == 0 else test_label)

    return df

In [44]:
N_A = 1000 #control
N_B = 1000 #test
bcr = 0.10  # baseline conversion rate
d_hat = 0.02  # difference between the groups

AB_data = generate_data(N_A, N_B, bcr, bcr+d_hat)

In [45]:
AB_data.head()

,group,converted
0,B,0
1,B,0
2,B,0
3,B,0
4,A,0


#### Difference in conversion rates

In [58]:
AB_data.groupby('group').apply(lambda x: x['converted'].sum()/len(x))
# 0.131394 - 0.084746 = 0.0466 different from 0!

group
A    0.084746
B    0.131394
dtype: float64

In [78]:
n1 = AB_data[AB_data.group == 'A'].converted.count()
x1 = AB_data[AB_data.group == 'A'].converted.sum()
p1 = x1/n1
n2 = AB_data[AB_data.group == 'B'].converted.count()
x2 = AB_data[AB_data.group == 'B'].converted.sum()
p2 = x2/n2
d_hat = p1-p2
n1, n2, p1, p2, d_hat

(1003, 997, 0.0847457627118644, 0.13139418254764293, -0.046648419835778526)

In [67]:
p = (p1*n1 + p2*n2) / (n1+n2) #pooled probability for H0
SE = np.sqrt((p*(1-p)/n1)+(p*(1-p)/n2)) #std dev for H0
p, SE

(0.108, 0.013880696454558993)

In [89]:
t_stat = d_hat/ SE #t statistic
critical_val = stats.norm.ppf(1-.975)
t_stat, critical_val #t_stat is in the rejection area so we are rejecting H0

(-3.3606685362287614, -1.959963984540054)

In [98]:
SE12 = np.sqrt((p1*(1-p1)/n1)+(p2*(1-p2)/n2))
dof = n1+n2-2
[d_hat - SE12 * stats.t.ppf(.975, dof), d_hat + SE12 * stats.t.ppf(.975, dof)], t_stat #far from confidence interval

([-0.07380914283806586, -0.0194876968334912], -3.3606685362287614)

In [107]:
pval = stats.t.cdf(t_stat, dof) * 2
pval #<0.05 so reject H0. there is statistical evidence against H0

0.0007921761170588799

In [76]:
#more automated:
from statsmodels.stats.proportion import proportions_ztest
c_ = np.array([c1, c2])
n_ = np.array([n1, n2])
stat, pval = proportions_ztest(c_, n_)
stat, pval, critical_val #again t_stat falls in the rejecting region. Also, pval<0.05 so reject H0, the two pop are different

(-3.3606685362287614, 0.0007775407646234423, -1.959963984540054)